# Run our models

Our training dataset and models are public on Hugging Face, you can get them directly.

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
from datasets import load_dataset
from tqdm.auto import tqdm

device = 0 if torch.cuda.is_available() else -1

c:\Users\soelm\Documents\04_Code\MVA\MVA_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Loading fine-tuned model

In [4]:
model = AutoModelForCausalLM.from_pretrained("SoelMgd/Poker_SmolLM")
tokenizer = AutoTokenizer.from_pretrained("HuggingFaceTB/SmolLM2-135M")
tokenizer.pad_token = tokenizer.eos_token

### Loading base model + LORA weights

Instead of using our fine-tuned model, you can also use the base model + LORA weights fine tuned

### Testing with a prompt

In [5]:
dataset = load_dataset("SoelMgd/Poker_Dataset")
test_example = dataset["test"][0]  # get the first test example

# Display the raw test example
print("Example:")
print("Question:", test_example["question"])
print("Ground Truth Answer:", test_example["answer"])

c:\Users\soelm\Documents\04_Code\MVA\MVA_venv\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\soelm\.cache\huggingface\hub\datasets--SoelMgd--Poker_Dataset. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Generating test split: 100%|██████████| 5243/5243 [00:00<00:00, 874313.61 examples/s]

Example:
Question: [TABLE_CONFIGURATION]
BTN=P2
SB=P3 0.5BB
BB=P4 1BB

[STACKS]
P1: 20.8BB
P2: 23.4BB [Kc 8c]
P3: 32.7BB
P4: 83.2BB
P5: 56.1BB
POT=1.5BB

[PREFLOP]
P5: FOLD
P1: FOLD
P2: RAISE 2BB
P3: CALL 1BB
P4: CALL 1BB

[STACKS]
P2: 21.4BB [Kc 8c]
P3: 31.2BB
P4: 82.2BB
POT=6.0BB

[FLOP][4d 7d Ah]
P3: CHECK
P4: CHECK
P2:
Ground Truth Answer: BET 1BB


In [7]:
# Generation pipeline
text_gen = pipeline("text-generation", model=model, tokenizer=tokenizer, return_full_text=False, device=device)

# Generate an answer using the pipeline
result = text_gen(test_example["question"], max_new_tokens=5, do_sample=False)
generated_answer = result[0]["generated_text"]

print("\nGenerated answer: ", generated_answer)
print("Ground Truth Answer:", test_example["answer"])

Device set to use cpu



Generated answer:   BET 2BB
Ground Truth Answer: BET 1BB
